<a href="https://colab.research.google.com/github/Tanay0320/Queuing-theory/blob/main/M_M_1_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import heapq
import numpy as np
import pandas as pd
from collections import deque

In [4]:
# Parameters
LAMBDA=2.0     # Arrival rate
MU=3.0     #Service rate
NUM_CUSTOMERS=1000    #Number of customers to simulate

In [7]:
# Initialization

np.random.seed(42)   #reproducibility

event_queue=[]  #Priority queue for event(time,type,customer_id)
queue=deque()   #FIFO queue of waiting customers
wait_times=[]   #record of waiting customers
customer_log=[]  #log of customer events

clock=0.0       #simulation clock
server_busy=False  #server state
next_customer_id=0    #customer ID generator


In [9]:
# Helper Functions

def schedule_arrival(curr_time):
  global next_customer_id
  interarrival=np.random.exponential(1/LAMBDA)
  arrival_time=curr_time+interarrival
  heapq.heappush(event_queue, (arrival_time,'arrival',next_customer_id))
  next_customer_id+=1

def schedule_departure(curr_time,customer_id):
  service_time=np.random.exponential(1/MU)
  departure_time=curr_time+service_time
  heapq.heappush(event_queue, (departure_time, 'departure',customer_id))
  return departure_time, service_time


In [15]:
# Start Simulation

schedule_arrival(clock)  #Schedule First Arrival

while len(wait_times) < NUM_CUSTOMERS:
  clock, event_type, cust_id=heapq.heappop(event_queue)

  if event_type=='arrival':
    if not server_busy:
      server_busy=True
      departure_time, service_time= schedule_departure(clock, cust_id)
      wait_times.append(0.0)
      customer_log.append({
            'customer_id':cust_id,
            'arrival_time':clock,
            'departure_time':departure_time,
            'wait_time':0.0,
            'service_time':service_time
      })
    else:
      queue.append((cust_id,clock))
    schedule_arrival(clock)

  elif event_type=='departure':
      if queue:
          next_id,arrival_time=queue.popleft()
          wait=clock-arrival_time
          wait_times.append(wait)
          departure_time, service_time=schedule_departure(clock,next_id)
          customer_log.append({
            'customer_id':next_id,
            'arrival_time':arrival_time,
            'start_service_time':clock,
            'departure_time':departure_time,
            'wait_time':wait,
            'service_time':service_time
          })

      else :
        server_busy=False

In [18]:
# Save to CSV
df=pd.DataFrame(customer_log)
df.to_csv("mm1_simulation_log.csv",index=False)
print("Simulation saved")

Simulation saved


In [17]:
# Print Summary Stats
print(f"Average waiting time: {np.mean(wait_times):.4f}")
print(f"Max waiting time: {np.max(wait_times):.4f}")
print(f"Total simulation time: {clock:.4f} units")

Average waiting time: 0.7248
Max waiting time: 6.2575
Total simulation time: 505.5969 units
